In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord
import warnings
import pickle
warnings.filterwarnings('ignore')

Define observing constraints: 

In [81]:
# Equilibrium temperatures
teq_min = 600 * u.K
teq_max = 1500 * u.K

rad_min = 2.0 * u.R_earth
rad_max = 5.0 * u.R_earth
gaia_mag_limit = 15

Download NASA Exoplanet Archive table from the web via astroquery, select the part of the population that is transiting and save it to a pickle file for running Pandexo simulations with Pandexo.py. WARNING: Do not run this script while Pandexo.py is running!

In [63]:
table = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)
transiting = table[(table['pl_tranflag'].data).astype(bool)]
pickle.dump( transiting, open( "table.p", "wb" ) )

After this line, you should run the accompanying script Pandexo.py to add Pandexo simulations of the SNR and observation time (minus overhead) as columns to the transiting planets table. This table is then read in again in the following line:

In [65]:
transiting = pickle.load(open("table_pandexo.p", "rb" ))

Compute equilibrium temperatures for zero albedo and the (relative) scale height: 

In [75]:
rp = transiting['pl_radj']
equilibrium_temperature = (transiting['st_teff'] * np.sqrt(transiting['st_rad'] / 2 / transiting['pl_orbsmax'])).decompose()
g = transiting['gaia_gmag'].quantity
transiting['teq'] = equilibrium_temperature

mean_molecular_mass = 6 * u.u#Assuming a default mean molecular weight. Unknown for almost all targets.
gp = const.G * 10*u.earthMass / rp**2#Assuming a default mass of 10Me. Most of our targets have no masses.
transiting['H'] = ((const.k_B * equilibrium_temperature) / (mean_molecular_mass * gp)).to(u.km)
transiting['signal'] = (2*rp*transiting['H']/transiting['st_rad']**2).decompose()
transiting['NIRSpec G395M_error'][transiting['NIRSpec G395M_error']<0]=np.nan#Set planets that were skipped by Pandexo to NaN.

Apply the selection constraints and prepare for output:

In [82]:
temp_constraints = (equilibrium_temperature < teq_max) & (equilibrium_temperature > teq_min)
rad_constraints = (rp < rad_max) & (rp > rad_min)
gmag_constaints = (g < gaia_mag_limit)
targets = transiting[rad_constraints & temp_constraints & gmag_constaints]#This is the table that will be plotted.

In [83]:
targets.sort('st_j')
targets['r_earth'] = targets['pl_radj'].to(u.R_earth)
targets['Teq']=np.round(targets['teq'],0)
targets['Rp']=np.round(targets['r_earth'],2)
targets['H ']=np.round(targets['H'],0)
targets['Orbital Period']=np.round(targets['pl_orbper'],2)
targets['DEC']=np.round(targets['dec'],1)
targets['duration']=np.round(targets['pl_trandur']*24.0,1)*u.h
targets['H_rel_depth_ppm']=np.round(1e6*targets['signal'],2)
targets['G395M_ppm']=np.round(1e6*targets['NIRSpec G395M_error'],1)
targets['SNR_H']=np.round(targets['signal']/targets['NIRSpec G395M_error'],4)
targets['T_eff']=np.round(targets['st_teff'],0)
targets[['pl_name', 'st_j', 'Teq', 'Rp', 'G395M_ppm' ,'pl_bmasse','Orbital Period','H ','H_rel_depth_ppm','SNR_H','duration','DEC','T_eff','st_spstr']].show_in_notebook()

idx,pl_name,st_j,Teq,Rp,G395M_ppm,pl_bmasse,Orbital Period,H,H_rel_depth_ppm,SNR_H,duration,DEC,T_eff,st_spstr
,,,K,earthRad,,,d,km,,,h,deg,K,
0,pi Men c,4.869,1167.0,2.04,47.5,4.82,6.27,69.0,3.05,0.0643,3.0,-80.5,6037.0,G0 V
1,HR 858 d,5.473,1068.0,2.16,58.9,--,11.23,71.0,2.35,0.0399,3.4,-30.8,6201.0,F6 V
2,HD 15337 c,7.553,644.0,2.39,175.3,8.11,17.18,52.0,4.41,0.0252,2.2,-27.6,5125.0,K1 V
3,HAT-P-11 b,7.608,829.0,4.36,nan,23.4,4.89,223.0,55.41,nan,0.0,48.1,4780.0,K4
4,GJ 9827 d,7.984,686.0,2.02,257.7,4.04,6.2,39.0,5.83,0.0226,1.2,-1.3,4340.0,K5 V
5,HD 106315 c,8.116,888.0,4.35,144.7,15.2,21.06,237.0,16.1,0.1113,4.6,-0.4,6327.0,F5 V
6,HD 106315 b,8.116,1155.0,2.44,160.5,12.6,9.55,98.0,3.72,0.0232,3.8,-0.4,6327.0,F5 V
7,Wolf 503 b,8.324,791.0,2.03,249.3,--,6.0,46.0,5.17,0.0207,1.3,-6.1,4716.0,K3.5 V
8,Kepler-410 A b,8.417,1004.0,2.84,nan,--,17.83,114.0,4.68,nan,0.0,45.1,6273.0,--


In [15]:
#table[['pl_name','pl_orbper']].show_in_notebook() #This outputs the entire table of exoplanets.